<a href="https://colab.research.google.com/github/AdirAmitaf-RidaFatima/Fraud-Detection-System-for-Transactions/blob/main/Fraud_Detection_System_for_Transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building logic for single Trasaction

In [2]:
# ----------------------------
# Fraud Detection System (Rule-Based)
# ----------------------------

from datetime import datetime

# Sample transaction dataset (you can expand this)
transactions = [
    {
        'id': 1,
        'user_id': 'U001',
        'amount': 150000,
        'location': 'Nigeria',
        'ip_address': '196.1.1.5',
        'merchant': 'XYZ Store',
        'timestamp': '2025-07-25 02:30:00',
        'account_created': '2025-07-20',
        'credit_limit': 150000,
        'currency': 'PKR',
        'status': 'Success'
    },
    # Add more transactions here
]

# ----------------------------
# RULE FUNCTIONS (Each Checks a Different Fraud Pattern)
# ----------------------------

def is_large_transaction(txn, threshold=100000):
    """Checks if transaction amount exceeds a certain threshold."""
    return txn['amount'] > threshold


def is_new_account(txn, min_days_old=7):
    """Checks if account is newer than X days."""
    acc_date = datetime.strptime(txn['account_created'], '%Y-%m-%d')
    txn_date = datetime.strptime(txn['timestamp'], '%Y-%m-%d %H:%M:%S')
    days_active = (txn_date - acc_date).days
    return days_active < min_days_old


def is_night_transaction(txn):
    """Checks if transaction happened between 12AM and 5AM."""
    txn_time = datetime.strptime(txn['timestamp'], '%Y-%m-%d %H:%M:%S').time()
    return txn_time.hour >= 0 and txn_time.hour <= 5


def is_foreign_location(txn, allowed_country='Pakistan'):
    """Checks if transaction location is foreign."""
    return txn['location'] != allowed_country


def is_round_amount(txn):
    """Detects if amount is round (e.g. 10000, 50000)."""
    return txn['amount'] % 1000 == 0


def is_blacklisted_merchant(txn, blacklist=['XYZ Store', 'Scam Mart']):
    """Checks if the merchant is blacklisted."""
    return txn['merchant'] in blacklist


def is_exact_credit_limit_usage(txn):
    """Checks if amount exactly equals credit limit (rarely natural)."""
    return txn['amount'] == txn['credit_limit']


def is_country_ip_mismatch(txn, ip_country='Pakistan'):
    """Simulates IP geolocation mismatch (for simplicity, based on hardcoded logic)."""
    if 'Nigeria' in txn['location'] and '196.' in txn['ip_address']:
        return True
    return False


def failed_then_success_pattern(txns):
    """
    Detects if a transaction followed a failed one with same amount.
    Requires list of transactions.
    """
    seen_failures = {}
    flags = []
    for txn in txns:
        key = (txn['user_id'], txn['amount'])
        if txn['status'] == 'Failed':
            seen_failures[key] = True
        elif txn['status'] == 'Success' and seen_failures.get(key):
            flags.append(txn['id'])
    return flags

# ----------------------------
# MASTER CHECK FUNCTION
# ----------------------------

def run_fraud_checks(txn):
    """Runs all rules on a single transaction and returns triggered rules."""
    rules_triggered = []

    if is_large_transaction(txn):
        rules_triggered.append('Large Amount')
    if is_new_account(txn):
        rules_triggered.append('New Account')
    if is_night_transaction(txn):
        rules_triggered.append('Night Transaction')
    if is_foreign_location(txn):
        rules_triggered.append('Foreign Location')
    if is_round_amount(txn):
        rules_triggered.append('Round Figure')
    if is_blacklisted_merchant(txn):
        rules_triggered.append('Blacklisted Merchant')
    if is_exact_credit_limit_usage(txn):
        rules_triggered.append('Exact Credit Limit')
    if is_country_ip_mismatch(txn):
        rules_triggered.append('Country-IP Mismatch')

    return rules_triggered

# ----------------------------
# RUN DETECTION ON ALL TRANSACTIONS
# ----------------------------

def run_all_detections(transactions):
    print("--- Fraud Detection Report ---")
    for txn in transactions:
        flags = run_fraud_checks(txn)
        if flags:
            print(f"Transaction ID {txn['id']} flagged for: {', '.join(flags)}")
        else:
            print(f"Transaction ID {txn['id']} appears normal.")

    # Also run failed-then-success pattern detection (batch-level rule)
    flagged_ids = failed_then_success_pattern(transactions)
    for txn_id in flagged_ids:
        print(f"Transaction ID {txn_id} flagged for: Failed-Then-Success Pattern")

# ----------------------------
# RUN THE ENGINE
# ----------------------------

run_all_detections(transactions)

--- Fraud Detection Report ---
Transaction ID 1 flagged for: Large Amount, New Account, Night Transaction, Foreign Location, Round Figure, Blacklisted Merchant, Exact Credit Limit, Country-IP Mismatch


# Multiple Transactions as Input

In [3]:
# --------------------------
# Fraud Detection System (Beginner Friendly)
# --------------------------
# This script simulates a simple fraud detection engine that flags suspicious financial transactions
# using various real-world-inspired rules and case studies.
# Each rule is coded as a separate function, and a main loop runs through all transactions to check.

# Sample transaction format (dictionary):
# {
#   "transaction_id": 1001,
#   "account_id": "ACC001",
#   "amount": 50000,
#   "location": "Karachi",
#   "transaction_type": "Online Transfer",
#   "time": "2023-07-21 02:30:00"
# }

import datetime

# Sample transaction dataset
transactions = [
    {"transaction_id": 1001, "account_id": "ACC001", "amount": 50000, "location": "Karachi", "transaction_type": "Online Transfer", "time": "2023-07-21 02:30:00"},
    {"transaction_id": 1002, "account_id": "ACC002", "amount": 500000, "location": "Lahore", "transaction_type": "ATM Withdrawal", "time": "2023-07-21 14:10:00"},
    {"transaction_id": 1003, "account_id": "ACC001", "amount": 3000000, "location": "Dubai", "transaction_type": "International Transfer", "time": "2023-07-21 02:50:00"},
    {"transaction_id": 1004, "account_id": "ACC003", "amount": 1500, "location": "Karachi", "transaction_type": "POS Purchase", "time": "2023-07-21 22:15:00"},
    {"transaction_id": 1005, "account_id": "ACC001", "amount": 60000, "location": "Karachi", "transaction_type": "Online Transfer", "time": "2023-07-21 02:35:00"},
]

# -----------------------------
# Rule 1: Threshold-Based Check
# -----------------------------
def check_high_amount(transaction, threshold=1000000):
    """Flags transactions over a given amount threshold (e.g., 1 million)"""
    if transaction['amount'] > threshold:
        return True, f"High amount transaction: {transaction['amount']}"
    return False, ""

# -----------------------------------------
# Rule 2: Suspicious Time (Odd-hour Activity)
# -----------------------------------------
def check_odd_hours(transaction):
    """Flags transactions occurring at suspicious hours (e.g., 12 AM - 5 AM)"""
    transaction_time = datetime.datetime.strptime(transaction['time'], "%Y-%m-%d %H:%M:%S")
    if transaction_time.hour < 5 or transaction_time.hour >= 23:
        return True, f"Odd hour transaction at {transaction_time.hour}h"
    return False, ""

# ---------------------------------------------------
# Rule 3: Rapid Repeated Transactions From Same Account
# ---------------------------------------------------
def check_rapid_transactions(transaction, all_transactions, window_minutes=10, count_threshold=2):
    """Flags if multiple transactions are made from same account within short time"""
    transaction_time = datetime.datetime.strptime(transaction['time'], "%Y-%m-%d %H:%M:%S")
    account_id = transaction['account_id']
    count = 0
    for tx in all_transactions:
        if tx['account_id'] == account_id and tx['transaction_id'] != transaction['transaction_id']:
            other_time = datetime.datetime.strptime(tx['time'], "%Y-%m-%d %H:%M:%S")
            diff = abs((transaction_time - other_time).total_seconds() / 60)
            if diff <= window_minutes:
                count += 1
    if count >= count_threshold:
        return True, f"Multiple transactions in {window_minutes} mins from {account_id}"
    return False, ""

# -------------------------------------------
# Rule 4: Foreign Transactions (Geo Anomaly)
# -------------------------------------------
def check_foreign_location(transaction, local_locations=["Karachi", "Lahore", "Islamabad"]):
    """Flags transactions from locations that are unusual for the user"""
    if transaction['location'] not in local_locations:
        return True, f"Foreign transaction location: {transaction['location']}"
    return False, ""

# -----------------------------------------------------
# Rule 5: Suspicious Transaction Type for Time of Day
# -----------------------------------------------------
def check_transaction_type_time(transaction):
    """Flags if certain types of transactions are done at suspicious times"""
    tx_type = transaction["transaction_type"]
    tx_time = datetime.datetime.strptime(transaction['time'], "%Y-%m-%d %H:%M:%S").hour
    if tx_type == "ATM Withdrawal" and (tx_time < 6 or tx_time > 22):
        return True, f"ATM withdrawal during odd hours ({tx_time}h)"
    return False, ""

# ------------------------------
# Master Fraud Detection Function
# ------------------------------
def run_fraud_detection(transactions):
    """Runs all fraud rules on each transaction"""
    flagged = []
    for tx in transactions:
        flags = []

        # Apply all rule checks
        checks = [
            check_high_amount(tx),
            check_odd_hours(tx),
            check_rapid_transactions(tx, transactions),
            check_foreign_location(tx),
            check_transaction_type_time(tx),
        ]

        # Collect any rule flags
        for result, reason in checks:
            if result:
                flags.append(reason)

        if flags:
            flagged.append({
                "transaction_id": tx['transaction_id'],
                "flags": flags
            })

    return flagged

# -------------------------------
# Run the Engine & Print Results
# -------------------------------
if __name__ == "__main__":
    flagged_transactions = run_fraud_detection(transactions)

    print("🚨 Flagged Suspicious Transactions:\n")
    for entry in flagged_transactions:
        print(f"Transaction ID {entry['transaction_id']}:")
        for reason in entry['flags']:
            print(f"  ⚠️ {reason}")
        print("-" * 50)


🚨 Flagged Suspicious Transactions:

Transaction ID 1001:
  ⚠️ Odd hour transaction at 2h
--------------------------------------------------
Transaction ID 1003:
  ⚠️ High amount transaction: 3000000
  ⚠️ Odd hour transaction at 2h
  ⚠️ Foreign transaction location: Dubai
--------------------------------------------------
Transaction ID 1005:
  ⚠️ Odd hour transaction at 2h
--------------------------------------------------


# **When Input is in CSV**
Your system need to get database that easily available.

In [9]:
# --------------------------
# Fraud Detection System (CSV-Based Input)
# --------------------------
# This version loads transactions from a CSV file
# and applies fraud detection rules.

import datetime
from google.colab import drive # Remove this if you are working in Github Code space
import pandas as pd

# -------------------------------
# Load Data from CSV File
# -------------------------------
# Ensure the CSV file has these columns:
# transaction_id, account_id, amount, location, transaction_type, time
drive.mount('/content/drive') # Dont ad thiss if in Codespace
CSV_FILE_PATH = "/content/sample_transactions.csv"  # Change to your actual file path

# Load CSV into DataFrame
transactions_df = pd.read_csv(CSV_FILE_PATH)

# Convert 'time' to datetime format
transactions_df['time'] = pd.to_datetime(transactions_df['time'])

# Convert DataFrame rows to list of dictionaries for compatibility with rule functions
transactions = transactions_df.to_dict(orient="records")

# -----------------------------
# Rule 1: Threshold-Based Check
# -----------------------------
def check_high_amount(transaction, threshold=1000000):
    """Flags transactions over a given amount threshold (e.g., 1 million)"""
    if transaction['amount'] > threshold:
        return True, f"High amount transaction: {transaction['amount']}"
    return False, ""

# -----------------------------------------
# Rule 2: Suspicious Time (Odd-hour Activity)
# -----------------------------------------
def check_odd_hours(transaction):
    """Flags transactions occurring at suspicious hours (e.g., 12 AM - 5 AM)"""
    transaction_time = transaction['time']
    if transaction_time.hour < 5 or transaction_time.hour >= 23:
        return True, f"Odd hour transaction at {transaction_time.hour}h"
    return False, ""

# ---------------------------------------------------
# Rule 3: Rapid Repeated Transactions From Same Account
# ---------------------------------------------------
def check_rapid_transactions(transaction, all_transactions, window_minutes=10, count_threshold=2):
    """Flags if multiple transactions are made from same account within short time"""
    transaction_time = transaction['time']
    account_id = transaction['account_id']
    count = 0
    for tx in all_transactions:
        if tx['account_id'] == account_id and tx['transaction_id'] != transaction['transaction_id']:
            other_time = tx['time']
            diff = abs((transaction_time - other_time).total_seconds() / 60)
            if diff <= window_minutes:
                count += 1
    if count >= count_threshold:
        return True, f"Multiple transactions in {window_minutes} mins from {account_id}"
    return False, ""

# -------------------------------------------
# Rule 4: Foreign Transactions (Geo Anomaly)
# -------------------------------------------
def check_foreign_location(transaction, local_locations=["Karachi", "Lahore", "Islamabad"]):
    """Flags transactions from locations that are unusual for the user"""
    if transaction['location'] not in local_locations:
        return True, f"Foreign transaction location: {transaction['location']}"
    return False, ""

# -----------------------------------------------------
# Rule 5: Suspicious Transaction Type for Time of Day
# -----------------------------------------------------
def check_transaction_type_time(transaction):
    """Flags if certain types of transactions are done at suspicious times"""
    tx_type = transaction["transaction_type"]
    tx_time = transaction['time'].hour
    if tx_type == "ATM Withdrawal" and (tx_time < 6 or tx_time > 22):
        return True, f"ATM withdrawal during odd hours ({tx_time}h)"
    return False, ""

# ------------------------------
# Master Fraud Detection Function
# ------------------------------
def run_fraud_detection(transactions):
    """Runs all fraud rules on each transaction"""
    flagged = []
    for tx in transactions:
        flags = []

        # Apply all rule checks
        checks = [
            check_high_amount(tx),
            check_odd_hours(tx),
            check_rapid_transactions(tx, transactions),
            check_foreign_location(tx),
            check_transaction_type_time(tx),
        ]

        # Collect any rule flags
        for result, reason in checks:
            if result:
                flags.append(reason)

        if flags:
            flagged.append({
                "transaction_id": tx['transaction_id'],
                "flags": flags
            })

    return flagged

# -------------------------------
# Run the Engine & Print Results
# -------------------------------
if __name__ == "__main__":
    flagged_transactions = run_fraud_detection(transactions)

    print("\n🚨 Flagged Suspicious Transactions:\n")
    for entry in flagged_transactions:
        print(f"Transaction ID {entry['transaction_id']}:")
        for reason in entry['flags']:
            print(f"  ⚠️ {reason}")
        print("-" * 50)

Mounted at /content/drive

🚨 Flagged Suspicious Transactions:

Transaction ID TXN1002:
  ⚠️ Odd hour transaction at 1h
  ⚠️ Foreign transaction location: Dubai
--------------------------------------------------
Transaction ID TXN1003:
  ⚠️ Odd hour transaction at 23h
  ⚠️ Foreign transaction location: New York
--------------------------------------------------
Transaction ID TXN1004:
  ⚠️ Foreign transaction location: Dubai
--------------------------------------------------
Transaction ID TXN1005:
  ⚠️ Foreign transaction location: Dubai
--------------------------------------------------
Transaction ID TXN1006:
  ⚠️ Odd hour transaction at 2h
--------------------------------------------------
Transaction ID TXN1009:
  ⚠️ Foreign transaction location: London
--------------------------------------------------
Transaction ID TXN1010:
  ⚠️ Foreign transaction location: Dubai
--------------------------------------------------
Transaction ID TXN1011:
  ⚠️ Foreign transaction location: New Yo